# Pandas數據分析

今天介紹資料分析近來很紅的 pandas 套件, 作者是 Wes McKinney。Python 會成為一個數據分析的熱門語言, 和 pandas 的出現也有相當的關係。

但是 pandas 雖然功能強, 但有些地方沒那麼直覺, 有時會讓大家以為是個深奧的套件。其實你大約可以把 pandas 想成「Python 的 Excel」, 但是功能更強、更有彈性、也有更多的可能性。

下面介紹個基本上就是把 pandas 當 Excel 學的影片, 相信大家會覺得很親切。<br>
https://youtu.be/9d5-Ti6onew

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## 1 開始使用 `pandas`

首先我們來讀入一個 CSV 檔, 這裡有個「假的」學測成績, 叫 `grades.csv` 我們來練習一下。

In [ ]:
grades = pd.read_csv("data/grades.csv")

用 `df` 是標準的叫法 (雖然這名稱我們隨便取也可以), 意思是 Data Frame, 這是 `pandas` 兩大資料結構之一。我們可以把 Data Frame 想成一張表格 (雖然其實可以是很多張表格)。

我們來看看我們 `df` 的前五筆資料。

In [ ]:
grades.head(n = 5)

如果你曾經手動讀入 CSV 檔, 就知道這省了多少事 (雖然我個人還挺喜歡純手動帶進 CSV)。

#### Excel 檔也可以快速讀入

不只 CSV 檔, 很多資料檔案, 像 Excel 檔都很容易在 `pandas` 完成。使用法是這樣:

    df2 = pd.read_excel('filename.xls', 'sheetname')
    
其中 sheetname 那裡要放工作表的名稱, 如果是中文的最好改成英文。

## 2 Pandas 基本資料結構

Pandas 有兩個基本資料結構:

* <b style="color:red;">DataFrame</b>: 可以想成一個表格。
* <b style="color:red;">Series</b>: 表格的某一列、某一行, 基本上就是我們以前的 list 或 array

一個 DataFrame, 我們有 `index` (列的名稱), `columns` (行的名稱)。

#### DataFrame

![DataFrame 的結構](images/indexcol.png)

#### Series

剛剛說 series 大概就是一個 list, 一個 array。其實更精準的說, 其實是一個有 "index" 的 array。

DataFrame 的每一行或每一列其實也都是一個 series。我們來看個例子, 例如所有同學的國文成績, 就是一個 series。

In [ ]:
grades["國文"]

在 Python 3 中, 我們終於可以和英文同步, 用這種很炫的方式叫出所有國文成績。

In [ ]:
grades.國文

#### 資料畫出來

要畫個圖很容易。

In [ ]:
grades.國文.plot()

當然, 在這個例子中, 其實畫 histogram 圖更有意義一點。

In [ ]:
grades.國文.hist(bins = 15)

## 3 一些基本的資料分析

算平均。

In [ ]:
grades.國文.mean()

算標準差。

In [ ]:
grades.國文.std()

不如就該算的都幫我們算算...

In [ ]:
grades.describe()

有時我們很愛看的相關係數矩陣。

In [ ]:
grades.corr(numeric_only = True)

只算兩科間的相關係數當然也可以。

In [ ]:
grades.國文.corr(grades.數學)

## 4 增加一行

### 【技巧】

我們增加一行, 加入總級分。

In [ ]:
grades["總級分"] = grades.sum(axis = 1, numeric_only = True)

### 【技巧】

有計算的當然也可以的。

In [ ]:
grades["加權"] = grades.國文 + grades.英文 + grades.數學*2

In [ ]:
# grades["加權1"] = grades[["國文","英文","數學"]].dot(pd.Series([1, 1, 2],index = ["國文","英文","數學"]))

## 5 排序和 index 重設

### 【重點】排序的方法

我們依總級分來排序。

In [ ]:
grades.sort_values("總級分", ascending = False).head(10)

### 【重點】排序的方法

加權分最高, 同分才看總級分

In [ ]:
grades2 = grades.sort_values(["加權","總級分"], ascending = [False,False])

### 【重點】重設 index

In [ ]:
grades2.index = range(1, grades2.shape[0] + 1)
grades2.head(10)

## 6 篩出我們要的資料

基本上和 NumPy 的 array 篩法很像。

### 【重點】

找出數學滿級分同學。

In [ ]:
grades2[grades2["數學"] == 15].head(10)

### 【重點】

找出數學和英文都滿級分的同學。要注意 `and` 要用 `&`, `or` 要用 `|`。每個條件一定要加弧號。

In [ ]:
grades2[(grades2["數學"] == 15) & (grades2["英文"] == 15)]

## 7 刪除一行或一列

### 【重點】刪掉一行

我們來刪掉總級分的那行。

In [ ]:
grades2.drop(["總級分"], axis = 1).head()

### 【重點】改變原有的 DataFrame

我們會發現 `pandas` 很多動作都沒有更改原有的 DataFrame, 真的要改要加入

    inplace=True

In [ ]:
grades2.drop(["總級分"], axis = 1, inplace = True)

### 【重點】刪掉一列

刪掉列就是指定要刪去的 index。

In [ ]:
grades2.drop(index = 5, axis = 0).head()

### 【重點】刪掉一列

通常刪掉符合條件的比較合理 (注意是找到要刪掉的部份, 再找出相對的 index)。

In [ ]:
grades2.drop(grades2[grades2.姓名 == "李士賢"].index, axis = 0).head()

## 8 真實股價資料

有個從 `Pandas` 獨立出來的套件叫 `pandas-datareader`, 幾經波折, 先是 Yahoo! 的財務資料不能用, 後來又是 Google 的資料不能用, 不過至少現在看來 Yahoo! 還可以使用。

安裝 `pandas-datareader` 就標準 `conda` 安裝:

    conda install pandas-datareader
    
如果裝過, 但很久沒更新就用:

    conda update pandas-datareader

### 【例子】 分析 Apple 股價

In [ ]:
# 為防止網路有問題, 我們把這個檔案以 aapl.csv 存起來, 可以這樣讀入。
df = pd.read_csv('data/aapl.csv', index_col = "Date")

In [ ]:
df.head()

#### 只要最後 300 個交易日!

In [ ]:
df.tail(300)

#### 20 日的移動平均

In [ ]:
df.Close.plot()

In [ ]:
df.Close.rolling(20).mean().plot()

#### 20 日和 60 日的移動平均

In [ ]:
df.Close.plot(legend=True)
df.Close.rolling(20).mean().plot(label="$MA_{20}$",legend=True)
df.Close.rolling(60).mean().plot(label="$MA_{60}$",legend=True)

#### 準備做預測

我們用個非常天真的模型...

![天真股票模型](images/naive.png)

網路上說這是線性的 (可能嗎)!


In [ ]:
y = df.Close.values[1:]
x = df.Close.values[:-1]

In [ ]:
plt.scatter(x, y)

哦, 真的有點像線性的, 我們之後用線性迴歸試試看。

## 9 手工打造一個 DataFrame*

有時我們用手工打造一個簡單的 DataFrame, 可以更理解整個結構。其實很容易, 一個 DataFrame 基本上就包含兩個主要部份:

* 資料本身: 通常一個二維陣列 (矩陣)
* 行、列的名稱

我們來個簡單的小例子。

In [ ]:
np.random.seed(123)
mydata = np.random.randn(4,3)

把行列的名字放進去, 就成一個 DataFrame。我們列的部份先讓 Python 自己產生。

In [ ]:
df2 = pd.DataFrame(mydata, columns = list("ABC"))

In [ ]:
df2

#### 兩個表格上下貼起來

我們再來生一個 DataFrame, 再「貼」起來。

In [ ]:
np.random.seed(123)
df3 = pd.DataFrame(np.random.randn(3,3), columns=list("ABC"))

In [ ]:
df3

In [ ]:
df4 = pd.concat([df2,df3], axis = 0)

前面我們弄得亂七八糟的 index 重設一下。

In [ ]:
df4.index = range(df4.shape[0])

In [ ]:
df4

#### 横向的貼

In [ ]:
df5 = pd.concat([df2,df3], axis = 1)

等等, 這大小好像不太對也可以嗎? 答案是可以的!

In [ ]:
df5

#### 大一點的例子

我們來做前面「假的」學測資料。首先要有「假的」同學名單, 如果有興趣產生很多名字, 可以用這個服務。

[中文姓名產生器](http://www.richyli.com/name/index.asp)

In [ ]:
names = pd.read_csv("data/names.csv", names = ["姓名"])

In [ ]:
names

In [ ]:
np.random.seed(123)
df_grades = pd.DataFrame(np.random.randint(6,16,(100,5)),
                        columns = ['國文','英文','數學','社會','自然'])

In [ ]:
df_grades

In [ ]:
pd.concat([names, df_grades], axis = 1)